In [1]:
from os import path
import graphlab as gl
from datetime import datetime

2016-03-23 11:58:27,300 [INFO] graphlab.deploy._session, 580: Using session dir: /Users/michael/Documents/turi-dev/local_scripts/../artifacts


# Load Data

MovieLens dataset collected by the GroupLens Research Project at the University of Minnesota.
For more information, see http://grouplens.org/datasets/movielens/

In [2]:
# Path to the dataset directory
data_dir = './dataset/ml-20m'

# Table of movies we are recommending: movieId, title, genres
items = gl.SFrame.read_csv(path.join(data_dir, 'movies.csv'))

# Table of interactions between users and items: userId, movieId, rating, timestamp
actions = gl.SFrame.read_csv(path.join(data_dir, 'ratings.csv'))

This commercial license of GraphLab Create is assigned to engr@turi.com.


2016-03-23 11:58:29,397 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: /tmp/graphlab_server_1458759508.log


Finished parsing file /Users/michael/Documents/sample-movie-recommender/dataset/ml-20m/movies.csv

Parsing completed. Parsed 100 lines in 0.08761 secs.

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


...

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,float,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 2042753 lines. Lines per second: 1.06306e+06

Read 13921441 lines. Lines per second: 1.91966e+06

Finished parsing file /Users/michael/Documents/sample-movie-recommender/dataset/ml-20m/ratings.csv

Parsing completed. Parsed 20000263 lines in 9.79849 secs.

# Prepare Data

In [3]:
# Prepare the data by removing items that are rare
rare_items = actions.groupby('movieId', gl.aggregate.COUNT).sort('Count')
rare_items = rare_items[rare_items['Count'] <= 5]
items = items.filter_by(rare_items['movieId'], 'movieId', exclude=True)
actions = actions.filter_by(rare_items['movieId'], 'movieId', exclude=True)
actions['timestamp'] = actions['timestamp'].astype(datetime)

In [4]:
# Extract year, title, and genre
items['year'] = items['title'].apply(lambda x: x[-5:-1])
items['title'] = items['title'].apply(lambda x: x[:-7])
items['genres'] = items['genres'].apply(lambda x: x.split('|'))

# Train Recommender Model

In [5]:
training_data, validation_data = gl.recommender.util.random_split_by_user(actions, 'userId', 'movieId')
model = gl.recommender.create(training_data, 'userId', 'movieId')

Recsys training: model = item_similarity

Warning: Ignoring columns rating, timestamp;

To use one of these as a target column, set target = <column_name>

and use a method that allows the use of a target.

Preparing data set.

...

| 15000           | 64.2387         |

| 16000           | 64.5681         |

| 17000           | 64.9078         |

+-----------------+-----------------+

Finished training in 68.2891s

# Explore and Evaluate the Model

In [6]:
# Get the metadata ready
urls = gl.SFrame.read_csv(path.join(data_dir, 'movie_urls.csv'))
items = items.join(urls, on='movieId')
users = gl.SFrame.read_csv(path.join(data_dir, 'user_names.csv'))

Finished parsing file /Users/michael/Documents/sample-movie-recommender/dataset/ml-20m/movie_urls.csv

Parsing completed. Parsed 100 lines in 0.079226 secs.

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/michael/Documents/sample-movie-recommender/dataset/ml-20m/movie_urls.csv

Parsing completed. Parsed 26863 lines in 0.055628 secs.

Finished parsing file /Users/michael/Documents/sample-movie-recommender/dataset/ml-20m/user_names.csv

Parsing completed. Parsed 100 lines in 0.033152 secs.

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/michael/Documents/sample-movie-recommender/dataset/ml-20m/user_names.csv

Parsing completed. Parsed 6743 lines in 0.017403 secs.

In [7]:
# Interactively evaluate and explore recommendations
view = model.views.overview(validation_set=validation_data,
                            user_data=users,
                            item_data=items,
                            item_name_column='title',
                            item_url_column='url')
view.show()

Tokenizing...

TF-IDF transform...

Creating inverted index...

Creating query expansion model...

Saving data for querying...

...

TF-IDF transform...

Creating inverted index...

Creating query expansion model...

Saving data for querying...

Getting similar items completed in 0.001925